In [16]:
from selenium import webdriver

In [17]:
import datetime, re, requests, io, time, random, string

In [18]:
from bs4 import BeautifulSoup

In [41]:
from credentials import email, password

In [20]:
driver = webdriver.Chrome('/Users/tannerhleigh/downloads/chromedriver')

In [21]:
# Go to wallmine.com
driver.get('https://wallmine.com')

In [30]:
if 'Make Smarter Investments' in driver.page_source:
    print('on the right page!')
    # Sign-in (copy x-path)
    sign_in_link = driver.find_element_by_xpath('/html/body/main/header/div/ul/li[1]/ul/li[3]/a')
    sign_in_link.click()
    time.sleep(3)
    print('Going to sign-in page')

In [38]:
if "Sign in here, please." in driver.page_source:
    login_email = driver.find_element_by_xpath('//*[@id="user_email"]')
    sign_in_password = driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[1]/div[2]')
    sign_in_password.click()
    time.sleep(0.2)

In [39]:
login_password = driver.find_element_by_xpath('//*[@id="user_password"]')
sign_in_button = driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[2]/div[1]/button')

In [42]:
login_email.send_keys(email)
login_password.send_keys(password)
sign_in_button.click()
time.sleep(3)

In [44]:
if 'Stock market overview' in driver.page_source:
    print('on the main page!')
    heatmap = driver.find_element_by_xpath('//*[@id="homepage-heatmap"]/a/div[2]')
    heatmap.click()
    time.sleep(2)

on the main page!


In [46]:
overview_tab = driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[1]/div/ul/li[1]/a')
overview_tab.click()
time.sleep(2)

In [52]:
if 'Free Stock Screen' in driver.page_source:
    print('on the stock tables!')
    stock_data = driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[2]').text
stock_list = stock_data.split('\n')

on the stock tables!


In [53]:
# finds the index point of the last table header
stock_list.index('Performance today')

27

In [56]:
# 28 is the index of the first company on the table
stock_list[28]
# stock_list[28:] to view the whole list

'AAPL Apple, Inc. NASDAQ Consumer Electronics $1.91T $74.542B 33.68 24.95 2.74 52.096M 62.12% 29 Oct 2020 $109.17 -2.36%'

In [63]:
company_info_parsed = stock_list[28].split(' $')[0]
market_cap = stock_list[28].split(' $')[1]
market_info = stock_list[28].split(' $')[2]
price_info = stock_list[28].split(' $')[3].split(' ')

price = f'${price_info[0]}'
performance_today = price_info[1]

['AAPL', 'Apple,', 'Inc.', 'NASDAQ', 'Consumer', 'Electronics']

In [79]:
company_info = company_info_parsed.split(' ')

if 'NASDAQ' in company_info: 
    idx = company_info.index('NASDAQ')
    exchange = company_info[idx]
#     print('here')
elif 'NYSE' in company_info:
    idx = company_info.index('NYSE')
    exchange = company_info[idx]
elif 'NYSEMKT' in company_info:
    idx = company_info.index('NYSEMKT')
    exchange = company_info[idx]

industry = " ".join(company_info[idx + 1:])
symbol = company_info[0]
company_name = " ".join(company_info[1:idx])
company_name

# Currently Parse
    # symbol, company name, price, exchange, industry, market_cap, performance_today
    # ebitda, p_e, ev_ebitda, debt_equity, average_volume, institutional_ownership, earning_date

'Apple, Inc.'

In [ ]:
# Parse market info
market_info.split(' ')

In [86]:
ebitda = market_info.split(' ')[0]
p_e = market_info.split(' ')[1]
ev_ebitda = market_info.split(' ')[2]
debt_equity = market_info.split(' ')[3]
average_volume = market_info.split(' ')[4]
institutional_ownership = market_info.split(' ')[5]
earnings_date = " ".join(market_info.split(' ')[6:])


In [87]:
x_stock = {
    "symbol": symbol,
    "company_name": company_name,
    "exchange": exchange,
    "market_cap": market_cap,
    "price": price,
    "performance_today": performance_today,
    "ebitda": ebitda,
    "p_e": p_e,
    "ev_ebitda": ev_ebitda,
    "average_volume": average_volume,
    "institutional_ownership": institutional_ownership,
    "earnings_date": earnings_date
}

x_stock

{'symbol': 'AAPL',
 'company_name': 'Apple, Inc.',
 'exchange': 'NASDAQ',
 'market_cap': '1.91T',
 'price': '$109.17',
 'performance_today': '-2.36%',
 'ebitda': '74.542B',
 'p_e': '33.68',
 'ev_ebitda': '24.95',
 'average_volume': '52.096M',
 'institutional_ownership': '62.12%',
 'earnings_date': '29 Oct 2020'}

In [96]:
def parse_stock_data(data):
    company_info_parsed = data.split(' $')[0]
    market_cap = data.split(' $')[1]
    market_info = data.split(' $')[2]
    price_info = data.split(' $')[3].split(' ')
    price = f'${price_info[0]}'
    performance_today = price_info[1]
    
    company_info = company_info_parsed.split(' ')

    if 'NASDAQ' in company_info: 
        idx = company_info.index('NASDAQ')
        exchange = company_info[idx]
    #     print('here')
    elif 'NYSE' in company_info:
        idx = company_info.index('NYSE')
        exchange = company_info[idx]
    elif 'NYSEMKT' in company_info:
        idx = company_info.index('NYSEMKT')
        exchange = company_info[idx]

    industry = " ".join(company_info[idx + 1:])
    symbol = company_info[0]
    company_name = " ".join(company_info[1:idx])
    
    ebitda = market_info.split(' ')[0]
    p_e = market_info.split(' ')[1]
    ev_ebitda = market_info.split(' ')[2]
    debt_equity = market_info.split(' ')[3]
    average_volume = market_info.split(' ')[4]
    institutional_ownership = market_info.split(' ')[5]
    earnings_date = " ".join(market_info.split(' ')[6:])
    
    result = {
        "symbol": symbol,
        "company_name": company_name,
        "exchange": exchange,
        "industry": industry,
        "market_cap": market_cap,
        "price": price,
        "performance_today": performance_today,
        "ebitda": ebitda,
        "p_e": p_e,
        "ev_ebitda": ev_ebitda,
        "average_volume": average_volume,
        "institutional_ownership": institutional_ownership,
        "earnings_date": earnings_date
    }
    
    return result

In [103]:
parse_stock_data(stock_list[30])

{'symbol': 'AMZN',
 'company_name': 'Amazon.com, Inc.',
 'exchange': 'NASDAQ',
 'industry': 'Specialty Retail',
 'market_cap': '1.51T',
 'price': '$3,053.73',
 'performance_today': '-2.41%',
 'ebitda': '30.457B',
 'p_e': '116.75',
 'ev_ebitda': '49.32',
 'average_volume': '5.062M',
 'institutional_ownership': '57.69%',
 'earnings_date': '22 Oct 2020'}

In [110]:
all_stocks = stock_list[28:]
# baba = all_stocks.pop(5)

In [111]:
for each_stock in all_stocks:
    print(parse_stock_data(each_stock))

{'symbol': 'AAPL', 'company_name': 'Apple, Inc.', 'exchange': 'NASDAQ', 'industry': 'Consumer Electronics', 'market_cap': '1.91T', 'price': '$109.17', 'performance_today': '-2.36%', 'ebitda': '74.542B', 'p_e': '33.68', 'ev_ebitda': '24.95', 'average_volume': '52.096M', 'institutional_ownership': '62.12%', 'earnings_date': '29 Oct 2020'}
{'symbol': 'MSFT', 'company_name': 'Microsoft Corp.', 'exchange': 'NASDAQ', 'industry': 'Internet Services & Infrastructure', 'market_cap': '1.54T', 'price': '$204.14', 'performance_today': '-1.58%', 'ebitda': '65.259B', 'p_e': '35.64', 'ev_ebitda': '22.40', 'average_volume': '34.993M', 'institutional_ownership': '74.09%', 'earnings_date': '22 Oct 2020'}
{'symbol': 'AMZN', 'company_name': 'Amazon.com, Inc.', 'exchange': 'NASDAQ', 'industry': 'Specialty Retail', 'market_cap': '1.51T', 'price': '$3,053.73', 'performance_today': '-2.41%', 'ebitda': '30.457B', 'p_e': '116.75', 'ev_ebitda': '49.32', 'average_volume': '5.062M', 'institutional_ownership': '57.

IndexError: list index out of range